In [1]:
import ee

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=JZdPdavInAx86v_IU04ohKYgLqhV9uv_6d54GxPtUbk&tc=KmxGRXmh1CjKDo85mippA9dTjwmUxCntUVU63RXbK0s&cc=7dWG4LbajRvKanOkBHznG9v1NbrEK7mDeMgktL4Uiz0

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AbUR2VNoituPMNBjMQdjUe0ujut4Up37xxNmU5FuhOhnpJodp8QCh46odvQ

Successfully saved authorization token.


In [ ]:
!pip install jdatetime

In [3]:
import datetime
from dateutil.relativedelta import relativedelta
import jdatetime

def jalali_to_gregorian(date):
    date_gregorian = jdatetime.date(year=date.year, month=date.month, day=date.day).togregorian()
    return date_gregorian.strftime("%Y-%m-%d")

# Define the AOI geometry
AOI = ee.Geometry.Polygon(
    [[[51.617697749999984, 36.46556635000002],
      [51.617697749999984, 36.64954727500002],
      [51.84710483333332, 36.64954727500002],
      [51.84710483333332, 36.46556635000002],
      [51.617697749999984, 36.46556635000002]]])

# Convert the end date to Gregorian
end_date_jalali = "1401-02-13"  # Replace with your Jalali end date
end_date = jdatetime.datetime.strptime(end_date_jalali, '%Y-%m-%d').date()
END_DATE = jalali_to_gregorian(end_date)

# Calculate the start date by subtracting 14 days from the end date

start_date = end_date - jdatetime.timedelta(days=14)

START_DATE = jalali_to_gregorian(start_date)

# Set the remaining parameters
CLOUD_FILTER = 50
CLD_PRB_THRESH = 50
NIR_DRK_THRESH = 0.15
CLD_PRJ_DIST = 1
BUFFER = 50
END_DATE, START_DATE

('2022-05-03', '2022-04-19')

In [4]:
# Import the folium library.
import folium

# Define a method for displaying Earth Engine image tiles to a folium map.
def add_ee_layer(self, ee_image_object, vis_params, name, show=True, opacity=1, min_zoom=0):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name=name,
        show=show,
        opacity=opacity,
        min_zoom=min_zoom,
        overlay=True,
        control=True
        ).add_to(self)

# Add the Earth Engine layer method to folium.
folium.Map.add_ee_layer = add_ee_layer

In [5]:
def add_cloud_bands(img):
    # Get s2cloudless image, subset the probability band.
    cld_prb = ee.Image(img.get('s2cloudless')).select('probability')

    # Condition s2cloudless by the probability threshold value.
    is_cloud = cld_prb.gt(CLD_PRB_THRESH).rename('clouds')

    # Add the cloud probability layer and cloud mask as image bands.
    return img.addBands(ee.Image([cld_prb, is_cloud]))

In [6]:
def add_cld_mask(img):
    # Add cloud component bands.
    img_cloud = add_cloud_bands(img)

    # Combine cloud and shadow mask, set cloud and shadow as value 1, else 0.
    is_cld = img_cloud.select('clouds').gt(0)
    # Remove small cloud-shadow patches and dilate remaining pixels by BUFFER input.
    # 20 m scale is for speed, and assumes clouds don't require 10 m precision.
    is_cld = (is_cld.focalMin(2).focalMax(BUFFER*2/20)
        .reproject(**{'crs': img.select([0]).projection(), 'scale': 20})
        .rename('cloudmask'))


    # Add the final cloud-shadow mask to the image.
    return img_cloud.addBands(is_cld)

In [7]:
def display_cloud_layers(col):
    # Mosaic the image collection.
    img = col.mosaic()
    # Subset layers and prepare them for display.
    clouds = img.select('clouds').selfMask()
    # Create a folium map object.
    center = AOI.centroid(10).coordinates().reverse().getInfo()
    m = folium.Map(location=center, zoom_start=12)

    # Add layers to the folium map.

    m.add_ee_layer(img,
                {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 2500, 'gamma': 1.1},
                'S2 image', True, 1, 9)

    m.add_ee_layer(clouds, {'palette': 'orange'},
                   'cloudmask', True, 0.5, 9)
    # Add a layer control panel to the map.
    m.add_child(folium.LayerControl())

    # Display the map.
    display(m)

In [8]:
def get_s2_sr_cld_col(aoi, start_date, end_date):
    # Import and filter S2 SR.
    s2_sr_col = (ee.ImageCollection('COPERNICUS/S2_SR')
        .filterBounds(aoi)
        .filterDate(start_date, end_date)
        .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', CLOUD_FILTER)))

    # Import and filter s2cloudless.
    s2_cloudless_col = (ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')
        .filterBounds(aoi)
        .filterDate(start_date, end_date))

    # Join the filtered s2cloudless collection to the SR collection by the 'system:index' property.
    return ee.ImageCollection(ee.Join.saveFirst('s2cloudless').apply(**{
        'primary': s2_sr_col,
        'secondary': s2_cloudless_col,
        'condition': ee.Filter.equals(**{
            'leftField': 'system:index',
            'rightField': 'system:index'
        })
    }))

In [9]:
s2_sr_cld_col_eval = get_s2_sr_cld_col(AOI, START_DATE, END_DATE)

In [97]:
def get_image_count(collection):
    # Get the number of images in the collection
    image_count = collection.size()

    # Return the image count as an integer
    return int(image_count.getInfo())

In [98]:
# Get the number of images in the collection
image_count = get_image_count(s2_sr_cld_col_eval)

# Print the number of images
print("Image Count:", image_count)

Image Count: 2


In [11]:
def get_band_names(collection):
    # Get the first image in the collection
    first_image = ee.Image(collection.first())

    # Get the band names of the first image
    band_names = first_image.bandNames()

    # Return the band names as a list
    return band_names.getInfo()

In [ ]:
s2_sr_cld_col_eval_disp = s2_sr_cld_col_eval.map(add_cld_mask)
get_band_names(s2_sr_cld_col_eval_disp)

In [13]:
display_cloud_layers(s2_sr_cld_col_eval_disp)

In [149]:
import datetime
from dateutil.relativedelta import relativedelta
import jdatetime

def jalali_to_gregorian(date):
    date_gregorian = jdatetime.date(year=date.year, month=date.month, day=date.day).togregorian()
    return date_gregorian.strftime("%Y-%m-%d")

# Define the AOI geometry
AOI = ee.Geometry.Polygon(
    [[[51.617697749999984, 36.46556635000002],
      [51.617697749999984, 36.64954727500002],
      [51.84710483333332, 36.64954727500002],
      [51.84710483333332, 36.46556635000002],
      [51.617697749999984, 36.46556635000002]]])

# Convert the end date to Gregorian
end_date_jalali = "1401-02-13"  # Replace with your Jalali end date
end_date = jdatetime.datetime.strptime(end_date_jalali, '%Y-%m-%d').date()
END_DATE = jalali_to_gregorian(end_date)

# Calculate the start date by subtracting 14 days from the end date

start_date = end_date - jdatetime.timedelta(days=14)

START_DATE = jalali_to_gregorian(start_date)

# Set the remaining parameters
CLOUD_FILTER = 50
CLD_PRB_THRESH = 20
NIR_DRK_THRESH = 0.15
CLD_PRJ_DIST = 1
BUFFER =100
END_DATE, START_DATE

('2022-05-03', '2022-04-19')

In [150]:
def apply_cld_mask(img):
    # Subset the cloudmask band and invert it so clouds/shadow are 0, else 1.
    not_cld = img.select('clouds').Not()

    # Subset reflectance bands and update their masks, return the result.
    return img.select('B.*').updateMask(not_cld)

In [151]:
s2_sr_cld_col = get_s2_sr_cld_col(AOI, START_DATE, END_DATE)

In [152]:
s2_sr_median = (s2_sr_cld_col.map(add_cld_mask)
                             .map(apply_cld_mask)
                             .median())

s2_sr_first = (s2_sr_cld_col.map(add_cld_mask)
                             .map(apply_cld_mask)
                             .first())



In [153]:
def get_least_cloudy_image(collection):
    # Sort the collection by cloud coverage in ascending order
    sorted_collection = collection.sort("CLOUDY_PIXEL_PERCENTAGE")

    # Get the first (least cloudy) image from the sorted collection
    least_cloudy_image = ee.Image(sorted_collection.first())

    # Return the least cloudy image
    return least_cloudy_image



# Get the least cloudy image from the collection
least_cloudy_image = get_least_cloudy_image(s2_sr_cld_col)

# Print the ID of the least cloudy image
print("Least Cloudy Image ID:", least_cloudy_image.id().getInfo())

Least Cloudy Image ID: 20220422T071609_20220422T072128_T39SWA


In [154]:
def display_cloud_layers(col):
    # Mosaic the image collection.
    img = col.mosaic()
    s2_sr_image = col.first()

    # Create a folium map object.
    center = AOI.centroid(10).coordinates().reverse().getInfo()
    m = folium.Map(location=center, zoom_start=12)

    #ADD layers
    m.add_ee_layer(img,
                {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 2500, 'gamma': 1.1},
                'S2 image mosaic', True, 1, 9)

    m.add_ee_layer(s2_sr_image,
                {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 2500, 'gamma': 1.1},
                'S2 image first', True, 1, 9)

    m.add_ee_layer(s2_sr_first,
                {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 2500, 'gamma': 1.1},
                'S2 cloud first', True, 1, 9)
    m.add_ee_layer(s2_sr_median,
                {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 2500, 'gamma': 1.1},
                'S2 cloud median', True, 1, 9)


    # Add a layer control panel to the map.
    m.add_child(folium.LayerControl())

    # Display the map.
    display(m)

In [155]:
display_cloud_layers(s2_sr_cld_col)